В Python версии 3.4, когда состоялся дебют библиотеки asyncio,  
она включала декораторы и синтаксис генератора yield from для определения сопрограмм. 

Сопрограмма – это метод, который можно приостановить,  
если имеется потенциально длительная задача,  
а затем возобновить, когда она завершится. 

В Python 3.5 в самом языке была  
реализована полноценная поддержка сопрограмм и  асинхронного  
программирования, для чего были добавлены ключевые слова async и  await. 

статьи  
https://habr.com/ru/articles/667630/  
https://aiohttp-kxepal-test.readthedocs.io/en/latest/web.html

#### Пример 1.1 синхронный код

In [1]:
import time

def fun1(x):
    print(x**2)
    time.sleep(2)
    print('fun1 завершена')

def fun2(x):
    print(x**0.5)
    time.sleep(2)
    print('fun2 завершена')

def main():
    fun1(4)
    fun2(4)

started = time.time()
main()
print(f'прошло {time.time() - started:.2f} сек.')

print('-'*20)
print(type(fun1))
print(type(fun1(4)))

16
fun1 завершена
2.0
fun2 завершена
прошло 4.01 сек.
--------------------
<class 'function'>
16
fun1 завершена
<class 'NoneType'>


In [4]:
#### Пример 1.2 Асинхронный код

In [7]:
import asyncio
import time

async def fun1(x):
    print(x**2)
    await asyncio.sleep(3)
    print('fun1 завершена')

async def fun2(x):
    print(x**0.5)
    await asyncio.sleep(3)
    print('fun2 завершена')

async def main():
    task1 = asyncio.create_task(fun1(4))
    task2 = asyncio.create_task(fun2(4))

    await task1
    await task2

started = time.time()
#asyncio.run(main())
await main()
print(f'прошло {time.time() - started:.2f} сек.')

print('-'*20)
print(type(fun1))
print(type(fun1(4)))

16
2.0
fun1 завершена
fun2 завершена
прошло 3.01 сек.
--------------------
<class 'function'>
<class 'coroutine'>


C:\Users\USER\AppData\Local\Temp\ipykernel_24120\3341478036.py:28: RuntimeWarning: coroutine 'fun1' was never awaited
  print(type(fun1(4)))


In [6]:
asyncio.get_running_loop()

<ProactorEventLoop running=True closed=False debug=False>

### **Корутина (сопрограмма)** дает интерпретатору возможность возобновить базовую функцию,  
### которая была приостановлена в месте размещения ключевого слова await.

P.S.: асинхронная (или корутинная) функция — это f, а корутина — f()



### Футуры и задачи (Futures and tasks)

In [11]:
import asyncio

async def fun1(x):
    print(x**2)
    await asyncio.sleep(2)
    print('fun1 завершена')

async def fun2(x):
    print(x**0.5)
    await asyncio.sleep(2)
    print('fun2 завершена')

async def main():
    task1 = asyncio.create_task(fun1(4))
    task2 = asyncio.create_task(fun2(4))

    print(type(task1))
    print(task1.__class__.__bases__)

    await task1
    await task2

await main()

<class '_asyncio.Task'>
(<class '_asyncio.Future'>,)
16
2.0
fun1 завершена
fun2 завершена


#### Футура 
(если совсем упрощенно) — это оболочка для некой асинхронной сущности,  
позволяющая выполнять ее "как бы одновременно" с другими асинхронными сущностями,  
переключаясь от одной сущности к другой в точках,  
обозначенных ключевым словом await

#### Задача 
— это частный случай футуры, предназначенный для оборачивания корутины.

In [26]:
import asyncio
import time

async def fun1(x):
    print(x**2)
    await asyncio.sleep(2)
    print('fun1 завершена')

async def fun2(x):
    print(x**0.5)
    await asyncio.sleep(2)
    print('fun2 завершена')

async def main():
    await fun1(4)
    await fun2(4)

started = time.time()
# asyncio.run(main())
await main()
print(f'прошло {time.time() - started:.2f} сек.')

16
fun1 завершена
2.0
fun2 завершена
прошло 4.01 сек.


### Вывод:
В asyncio.run нужно передавать асинхронную функцию с эвейтами на задачи,  
а не на корутины. Иначе не будет конкурентности.

### Асинхронные менеджеры контекста

In [12]:
import asyncio

# имитация  асинхронного соединения с некой периферией
async def get_conn(host, port):
    class Conn:
        async def put_data(self):
            print('Отправка данных...')
            await asyncio.sleep(2)
            print('Данные отправлены.')

        async def get_data(self):
            print('Получение данных...')
            await asyncio.sleep(2)
            print('Данные получены.')

        async def close(self):
            print('Завершение соединения...')
            await asyncio.sleep(2)
            print('Соединение завершено.')

    print('Устанавливаем соединение...')
    await asyncio.sleep(2)
    print('Соединение установлено.')
    return Conn()


class Connection:
    # этот конструктор будет выполнен в заголовке with
    def __init__(self, host, port):
        self.host = host
        self.port = port

    # этот метод будет неявно выполнен при входе в with
    async def __aenter__(self):
        self.conn = await get_conn(self.host, self.port)
        return self.conn

    # этот метод будет неявно выполнен при выходе из with
    async def __aexit__(self, exc_type, exc, tb):
        await self.conn.close()


async def main():
    async with Connection('localhost', 9001) as conn:
        send_task = asyncio.create_task(conn.put_data())
        receive_task = asyncio.create_task(conn.get_data())

        # операции отправки и получения данных выполняем конкурентно
        await send_task
        await receive_task


#asyncio.run(main())
await main()

Устанавливаем соединение...
Соединение установлено.
Отправка данных...
Получение данных...
Данные отправлены.
Данные получены.
Завершение соединения...
Соединение завершено.


### Приложение ПОГОДА

In [13]:
!pip install aiohttp

In [14]:
import asyncio
import time
from aiohttp import ClientSession


async def get_weather(city):
    async with ClientSession() as session:
        url = f'http://api.openweathermap.org/data/2.5/weather'
        params = {'q': city, 'APPID': '2a4ff86f9aaa70041ec8e82db64abf56'}

        async with session.get(url=url, params=params) as response:
            weather_json = await response.json()
            print(f'{city}: {weather_json["weather"][0]["main"]}')


async def main(cities_):
    tasks = []
    for city in cities_:
        tasks.append(asyncio.create_task(get_weather(city)))

    for task in tasks:
        await task


cities = ['Moscow', 'St. Petersburg', 'Rostov-on-Don', 'Kaliningrad', 'Vladivostok',
          'Minsk', 'Beijing', 'Delhi', 'Istanbul', 'Tokyo', 'London', 'New York']

started = time.time()

# asyncio.run(main(cities))
await main(cities)

print(f'прошло {time.time() - started:.2f} сек.')

Moscow: Clouds
Vladivostok: Clouds
Kaliningrad: Clear
Delhi: Haze
Istanbul: Clouds
Beijing: Clouds
London: Clouds
St. Petersburg: Clouds
Minsk: Clouds
Tokyo: Clouds
Rostov-on-Don: Clear
New York: Clear
прошло 0.15 сек.


#### пробуем то же, но синхронно

In [15]:
import time
import requests

def get_weather(city):
    url = f'http://api.openweathermap.org/data/2.5/weather'
    params = {'q': city, 'APPID': '2a4ff86f9aaa70041ec8e82db64abf56'}

    weather_json = requests.get(url=url, params=params).json()
    print(f'{city}: {weather_json["weather"][0]["main"]}')

def main(cities_):
    for city in cities_:
        get_weather(city)

cities = ['Moscow', 'St. Petersburg', 'Rostov-on-Don', 'Kaliningrad', 'Vladivostok',
          'Minsk', 'Beijing', 'Delhi', 'Istanbul', 'Tokyo', 'London', 'New York']

started = time.time()
main(cities)
print(f'прошло {time.time() - started:.2f} сек.')

Moscow: Clouds
St. Petersburg: Clouds
Rostov-on-Don: Clear
Kaliningrad: Clear
Vladivostok: Clouds
Minsk: Clouds
Beijing: Clouds
Delhi: Haze
Istanbul: Clouds
Tokyo: Clouds
London: Clouds
New York: Clear
прошло 1.40 сек.


### Работа с результатом асинх функций
для группового запуска задач необходимо использовать уже не цикл с await, а функцию asyncio.gather

In [17]:
import asyncio
import time
from aiohttp import ClientSession

async def get_weather(city):
    async with ClientSession() as session:
        url = f'http://api.openweathermap.org/data/2.5/weather'
        params = {'q': city, 'APPID': '2a4ff86f9aaa70041ec8e82db64abf56'}

        async with session.get(url=url, params=params) as response:
            weather_json = await response.json()
            return f'{city}: {weather_json["weather"][0]["main"]}'

async def main(cities_):
    tasks = []
    for city in cities_:
        tasks.append(asyncio.create_task(get_weather(city)))

    results = await asyncio.gather(*tasks)

    for result in results:
        print(result)

cities = ['Moscow', 'St. Petersburg', 'Rostov-on-Don', 'Kaliningrad', 'Vladivostok',
          'Minsk', 'Beijing', 'Delhi', 'Istanbul', 'Tokyo', 'London', 'New York']

started = time.time()
# asyncio.run(main(cities))
await main(cities)
print(f'прошло {time.time() - started:.2f} сек.')

Moscow: Clouds
St. Petersburg: Clouds
Rostov-on-Don: Clear
Kaliningrad: Clear
Vladivostok: Clouds
Minsk: Clouds
Beijing: Clouds
Delhi: Haze
Istanbul: Clouds
Tokyo: Clouds
London: Clouds
New York: Clear
прошло 0.13 сек.


## Веб-сервер aiohttp

#### 1. Базовый веб-сервер

In [18]:
import asyncio
import json
from aiohttp import ClientSession, web


async def get_weather(city):
    async with ClientSession() as session:
        url = f'http://api.openweathermap.org/data/2.5/weather'
        params = {'q': city, 'APPID': '2a4ff86f9aaa70041ec8e82db64abf56'}

        async with session.get(url=url, params=params) as response:
            weather_json = await response.json()
            try:
                return weather_json["weather"][0]["main"]
            except KeyError:
                return 'Нет данных'


async def handle(request):
    city = request.rel_url.query['city']
    weather = await get_weather(city)
    result = {'city': city, 'weather': weather}

    return web.Response(text=json.dumps(result, ensure_ascii=False))


async def main():
    app = web.Application()
    app.add_routes([web.get('/weather', handle)])
    runner = web.AppRunner(app)
    await runner.setup()
    site = web.TCPSite(runner, 'localhost', 8080)
    await site.start()

    while True:
        await asyncio.sleep(3600)


# if __name__ == '__main__':
#     asyncio.run(main())

await main()

# http://localhost:8080/weather?city=Sochi
# http://localhost:8080/weather?city=Moscow

CancelledError: 

#### 2. Добавим остановку

In [3]:
# ayncio-1.py


import asyncio
import json
import signal
from aiohttp import ClientSession, web


async def get_weather(city):
    async with ClientSession() as session:
        url = f'http://api.openweathermap.org/data/2.5/weather'
        params = {'q': city, 'APPID': '2a4ff86f9aaa70041ec8e82db64abf56'}

        async with session.get(url=url, params=params) as response:
            weather_json = await response.json()
            try:
                return weather_json["weather"][0]["main"]
            except KeyError:
                return 'Нет данных'


async def handle(request):
    city = request.rel_url.query['city']
    weather = await get_weather(city)
    result = {'city': city, 'weather': weather}

    return web.Response(text=json.dumps(result, ensure_ascii=False))


async def main(host='127.0.0.1', port=8080):
    print(f'Старт сервера на {host}:{port}')
    app = web.Application()
    app.add_routes([web.get('/weather', handle)])
    runner = web.AppRunner(app)
    await runner.setup()
    site = web.TCPSite(runner, host, port)
    await site.start()


# if __name__ == '__main__':
#     loop = asyncio.get_event_loop()
#     loop.run_until_complete(main())
#     try:
#         loop.run_forever()
#     except KeyboardInterrupt:
#         pass


await main()


# http://localhost:8080/weather?city=Sochi
# http://localhost:8080/weather?city=Moscow

Старт сервера на 127.0.0.1:8080


#### Добавим логирование

In [1]:
!pip install aiologger

ERROR: Exception:
Traceback (most recent call last):
  File "/home/alex/dev/mvp/.venv-harvester/lib/python3.10/site-packages/pip/_vendor/urllib3/response.py", line 438, in _error_catcher
    yield
  File "/home/alex/dev/mvp/.venv-harvester/lib/python3.10/site-packages/pip/_vendor/urllib3/response.py", line 561, in read
    data = self._fp_read(amt) if not fp_closed else b""
  File "/home/alex/dev/mvp/.venv-harvester/lib/python3.10/site-packages/pip/_vendor/urllib3/response.py", line 527, in _fp_read
    return self._fp.read(amt) if amt is not None else self._fp.read()
  File "/home/alex/dev/mvp/.venv-harvester/lib/python3.10/site-packages/pip/_vendor/cachecontrol/filewrapper.py", line 98, in read
    data: bytes = self.__fp.read(amt)
  File "/usr/lib/python3.10/http/client.py", line 466, in read
    s = self.fp.read(amt)
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
  File "/usr/lib/python3.10/ssl.py", line 1303, in recv_into
    retur

In [ ]:
# ayncio-2.py


import asyncio
import json
import signal
from aiohttp import ClientSession, web
from aiologger.loggers.json import JsonLogger


logger = JsonLogger.with_default_handlers(
    level='DEBUG',
    serializer_kwargs={'ensure_ascii': False},
)


async def get_weather(city):
    async with ClientSession() as session:
        url = f'http://api.openweathermap.org/data/2.5/weather'
        params = {'q': city, 'APPID': '2a4ff86f9aaa70041ec8e82db64abf56'}

        async with session.get(url=url, params=params) as response:
            weather_json = await response.json()
            try:
                return weather_json["weather"][0]["main"]
            except KeyError:
                return 'Нет данных'


async def handle(request):
    city = request.rel_url.query['city']
    await logger.info(f'Получаем погоду для города {city}')
    weather = await get_weather(city)
    result = {'city': city, 'weather': weather}

    return web.Response(text=json.dumps(result, ensure_ascii=False))


async def main(host='127.0.0.1', port=8080):
    await logger.info(f'Старт сервера на {host}:{port}')
    app = web.Application()
    app.add_routes([web.get('/weather', handle)])
    runner = web.AppRunner(app)
    await runner.setup()
    site = web.TCPSite(runner, host, port)
    await site.start()


if __name__ == '__main__':
    loop = asyncio.get_event_loop()
    loop.run_until_complete(main())
    try:
        loop.run_forever()
    except KeyboardInterrupt:
        #await logger.info('Остановка сервера')
        pass


# await main()


# http://localhost:8080/weather?city=Sochi
# http://localhost:8080/weather?city=Moscow

#### 3. Дюбавим обращение к БД

In [ ]:
!pip install aiosqlite

In [ ]:
# ayncio-3.py


import asyncio
import json
import signal
from aiohttp import ClientSession, web
from aiologger import Logger
from aiologger.loggers.json import JsonLogger
import aiosqlite
from datetime import datetime


logger = Logger.with_default_handlers(name='my-logger')
# logger = JsonLogger.with_default_handlers(
#     level='DEBUG',
#     serializer_kwargs={'ensure_ascii': False},
# )


async def get_weather(city):
    async with ClientSession() as session:
        url = f'http://api.openweathermap.org/data/2.5/weather'
        params = {'q': city, 'APPID': '2a4ff86f9aaa70041ec8e82db64abf56'}

        async with session.get(url=url, params=params) as response:
            weather_json = await response.json()
            try:
                return weather_json["weather"][0]["main"]
            except KeyError:
                return 'Нет данных'


async def create_table():
    async with aiosqlite.connect('weather.db') as db:
        await db.execute('CREATE TABLE IF NOT EXISTS requests '
                         '(date text, city text, weather text)')
        await db.commit()


async def save_to_db(city, weather):
    async with aiosqlite.connect('weather.db') as db:
        await db.execute('INSERT INTO requests VALUES (?, ?, ?)',
                         (datetime.now(), city, weather))
        await db.commit()
        await logger.info(f'Сохранили в БД {city=}, {weather=}')


async def handle(request):
    city = request.rel_url.query['city']
    await logger.info(f'Получаем погоду для города {city}')
    weather = await get_weather(city)
    result = {'city': city, 'weather': weather}

    await save_to_db(city, weather)

    return web.Response(text=json.dumps(result, ensure_ascii=False))


async def main(host='127.0.0.1', port=8080):
    await create_table()

    await logger.info(f'Старт сервера на {host}:{port}')
    app = web.Application()
    app.add_routes([web.get('/weather', handle)])
    runner = web.AppRunner(app)
    await runner.setup()
    site = web.TCPSite(runner, host, port)
    await site.start()


if __name__ == '__main__':
    loop = asyncio.get_event_loop()
    loop.run_until_complete(main())
    try:
        loop.run_forever()
    except KeyboardInterrupt:
        logger.info('Остановка сервера')
        pass


# await main()


# http://localhost:8080/weather?city=Sochi
# http://localhost:8080/weather?city=Moscow